In [15]:
pip install tensorflow keras-facenet


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10387 sha256=953aecb3ce6d2f5fa2d04c73cc91571885e9ded53abcf894b1f3845a7a9996f0
  Stored in directory: c:\users\aayush the great\appdata\local\pip\cache\wheels\05\b0\f5\19ac49fedc10b1df3ee56b096edbcfa39d45794fccc6bcdbbf
Successfully built keras-facenet
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np
from keras_facenet import FaceNet
    
      
embedder = FaceNet()
    
dataset_path = "picuus"
    
known_face_encodings = []
known_face_names = []
    
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (160, 160))  
        face_encoding = embedder.embeddings([image])[0]  

        known_face_encodings.append(face_encoding)
        known_face_names.append(os.path.splitext(filename)[0])  

print(f"Loaded {len(known_face_names)} faces from dataset")

mp_face_detection = mp.solutions.face_detection

video_capture = cv2.VideoCapture(0)
final_detected_name = "Unknown" 

with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7) as face_detection:
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)

        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                x, y, w, h = int(bbox.xmin * w), int(bbox.ymin * h), int(bbox.width * w), int(bbox.height * h)

                
                x, y, w, h = max(x, 0), max(y, 0), min(w, frame.shape[1] - x), min(h, frame.shape[0] - y)
                face = frame[y:y+h, x:x+w]

                if face.size == 0:
                    continue  

                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (160, 160))  
                face_encoding = embedder.embeddings([face])[0]

                
                name = "Unknown"
                if len(known_face_encodings) > 0:
                    distances = np.linalg.norm(known_face_encodings - face_encoding, axis=1)
                    best_match = np.argmin(distances)

                    print(f"Distances: {distances}")  
                    print(f"Best match distance: {distances[best_match]}")

                    if distances[best_match] < 1.1:  # Adjusted threshold
                        name = known_face_names[best_match]

              
                final_detected_name = name

            
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow("Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

video_capture.release()
cv2.destroyAllWindows()

print(f"Final detected person: {final_detected_name}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━

In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np
from keras_facenet import FaceNet
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import CLSCTX_ALL

embedder = FaceNet()
dataset_path = "picuus"

known_face_encodings = []
known_face_names = []

for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (160, 160))  
        face_encoding = embedder.embeddings([image])[0]  
        known_face_encodings.append(face_encoding)
        known_face_names.append(os.path.splitext(filename)[0])

print(f"Loaded {len(known_face_names)} faces from dataset")


mp_face_detection = mp.solutions.face_detection
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

video_capture = cv2.VideoCapture(0)
final_detected_name = "Unknown"
is_verified = False  

with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7) as face_detection:
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)

        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                
                
                box_x = int(bbox.xmin * w)
                box_y = int(bbox.ymin * h)
                box_w = int(bbox.width * w)
                box_h = int(bbox.height * h)

               
                box_x, box_y, box_w, box_h = max(box_x, 0), max(box_y, 0), min(box_w, frame.shape[1] - box_x), min(box_h, frame.shape[0] - box_y)
                face = frame[box_y:box_y+box_h, box_x:box_x+box_w]
                
                if face.shape[0] == 0 or face.shape[1] == 0:
                    continue
                
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (160, 160))  
                face_encoding = embedder.embeddings([face])[0]
                
                name = "Unknown"
                if len(known_face_encodings) > 0:
                    distances = np.linalg.norm(np.array(known_face_encodings) - face_encoding, axis=1)
                    best_match = np.argmin(distances)

                    if distances[best_match] < 1.1: 
                        name = known_face_names[best_match]
                        is_verified = name == "Aayush" 

                final_detected_name = name

                cv2.rectangle(frame, (box_x, box_y), (box_x + box_w, box_y + box_h), (0, 255, 0), 2)
                cv2.putText(frame, name, (box_x, box_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        if is_verified:
            hand_results = hands.process(rgb_frame)
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    x_thumb = hand_landmarks.landmark[4].x
                    y_thumb = hand_landmarks.landmark[4].y
                    x_index = hand_landmarks.landmark[8].x
                    y_index = hand_landmarks.landmark[8].y
                    
                    pinch_distance = np.linalg.norm(np.array([x_thumb, y_thumb]) - np.array([x_index, y_index]))

                
                    current_volume = volume.GetMasterVolumeLevelScalar() or 0.5
                    new_volume = current_volume  

                    if pinch_distance < 0.05: 
                        new_volume = 0.5
                        volume.SetMasterVolumeLevelScalar(new_volume, None)
                    elif y_index < y_thumb:  
                        new_volume = min(1.0, current_volume + 0.05)
                        volume.SetMasterVolumeLevelScalar(new_volume, None)
                    elif y_index > y_thumb:
                        new_volume = max(0.0, current_volume - 0.05)
                        volume.SetMasterVolumeLevelScalar(new_volume, None)

                    cv2.putText(frame, f"Volume: {int(new_volume * 100)}%", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                    cv2.rectangle(frame, (50, 70), (50 + int(new_volume * 200), 100), (0, 255, 0), -1)

     
        cv2.imshow("Face & Hand Control", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

video_capture.release()
cv2.destroyAllWindows()

print(f"Final detected person: {final_detected_name}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━

In [1]:
pip install pycaw


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
